<a href="https://colab.research.google.com/github/guhang987/federated-learning/blob/master/fl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade tensorflow_federated_nightly
!pip install --quiet --upgrade nest_asyncio
import nest_asyncio
nest_asyncio.apply()
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

# TODO(b/148678573,b/148685415): must use the reference context because it
# supports unbounded references and tff.sequence_* intrinsics.
tff.backends.reference.set_reference_context()

     |████████████████████████████████| 512kB 3.4MB/s 
     |████████████████████████████████| 393.5MB 40kB/s 
     |████████████████████████████████| 112kB 51.8MB/s 
     |████████████████████████████████| 3.0MB 44.8MB/s 
     |████████████████████████████████| 1.1MB 45.0MB/s 
     |████████████████████████████████| 174kB 44.7MB/s 
     |████████████████████████████████| 153kB 51.0MB/s 
     |████████████████████████████████| 1.3MB 45.9MB/s 
     |████████████████████████████████| 10.6MB 45.7MB/s 
     |████████████████████████████████| 471kB 46.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: tf-nightly 2.4.0.dev20201020 has requirement absl-py~=0.10, but you'll have absl-py 0.9.0 which is incompatible.
ERROR: tf-nightly 2.4.0.dev20201020 has requirement grpcio~=1.32.0, but you'll have grpcio 1.29.0 which is incompatible.
ERROR: tf-nightly 2.4.0.dev20201020 has requirement numpy~=1.19.2, but you'll have numpy 1

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.4.0-dev20201020). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [ ]:
import tensorflow as tf
tf.__version__
# %load_ext tensorboard
# logdir = "/tmp/logs/scalars/training/"
# summary_writer = tf.summary.create_file_writer(logdir)

#with summary_writer.as_default():
  #for round_num in range(1, NUM_ROUNDS):
    #tf.summary.scalar(name, value, step=round_num)

#%tensorboard --logdir /tmp/logs/scalars/ --port=0

'2.4.0-dev20201020'

In [ ]:
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
import copy
#深拷贝
ordered_mnist_train=copy.deepcopy(mnist_train)
#给mnist_train按标签排序
f=0
for digit in range(10):
  index=[i for i,d in enumerate(mnist_train[1]) if d==digit]
  for i in range(5420):
    ordered_mnist_train[0][f+i]=mnist_train[0][index[i]]
    ordered_mnist_train[1][f+i]=mnist_train[1][index[i]]
  f+=5420

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
#检测排序结果
from matplotlib import pyplot as plt
for i in range(5420*3-1,5420*3+1):
  plt.imshow(np.array(ordered_mnist_train[0][i]), cmap='gray', aspect='equal')
  plt.grid(False)
  print(ordered_mnist_train[1][i])
  _ = plt.show()

In [ ]:
BATCH_SIZE = 100
# 定义：20个shard 每个shard包含271张图片
SHARD_SIZE = 271
def get_data(source,i,j):
  device=[]
  #根据普通索引ij计算对应shard数目，继而计算对应图片索引
  shard1=(20*j+i)*SHARD_SIZE
  shard2=(j+20*i+10)*SHARD_SIZE
  #600张图片索引存放在all_samples中
  all_samples=list(range(shard1,shard1+SHARD_SIZE))+list(range(shard2,shard2+SHARD_SIZE))

  batch_nums=int(2*SHARD_SIZE/BATCH_SIZE)
  #分成batch_nums个batch
  for i in range(int(batch_nums)):
    device_index=all_samples[BATCH_SIZE*i:BATCH_SIZE*i + BATCH_SIZE]
    device.append({
        'x':
            np.array([source[0][i].flatten() / 255.0 for i in device_index],
                    dtype=np.float32),
        'y':
            np.array([source[1][i] for i in device_index], dtype=np.int32)
    })
  return device
#构造联邦数据集
federated_train_data = [get_data(ordered_mnist_train,i,j) for i in range(10) for j in range(10)]

In [ ]:
#定义模型
BATCH_SPEC = collections.OrderedDict(
    x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
    y=tf.TensorSpec(shape=[None], dtype=tf.int32))
BATCH_TYPE = tff.to_type(BATCH_SPEC)
MODEL_SPEC = collections.OrderedDict(
    weights=tf.TensorSpec(shape=[784, 10], dtype=tf.float32),
    bias=tf.TensorSpec(shape=[10], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

import random
@tf.function
def forward_pass(model, batch):
  #L2正则项
  loss=10e-4*tf.reduce_sum(model['weights']*model['weights'])+tf.reduce_sum(model['bias']*model['bias'])/(2*BATCH_SIZE)
  #(100, 784) *(784, 10)+ (10,)=(100,10) 即100个预测值
  predicted_y = tf.nn.softmax(
      tf.matmul(batch['x'], model['weights']) + model['bias'])
  #返回这一整个batch的损失值
  return tf.reduce_mean(
      #记录100个图片的损失值
          tf.nn.softmax_cross_entropy_with_logits(
          #tf.math.log(predicted_y)是100*10矩阵
          #对应元素相乘
          tf.one_hot(batch['y'], 10),tf.math.log(predicted_y)))+loss

#批训练
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
  return forward_pass(model, batch)

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
  # Define a group of model variables and set them to `initial_model`. Must
  # be defined outside the @tf.function.
  model_vars = collections.OrderedDict([
      (name, tf.Variable(name=name, initial_value=value))
      for name, value in initial_model.items()
  ])
  optimizer = tf.keras.optimizers.SGD(learning_rate)

  @tf.function
  def _train_on_batch(model_vars, batch):
    # Perform one step of gradient descent using loss from `batch_loss`.
    with tf.GradientTape() as tape:
      loss = forward_pass(model_vars, batch)
    grads = tape.gradient(loss, model_vars)
    optimizer.apply_gradients(
        zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
    return model_vars

  return _train_on_batch(model_vars, batch)

#设备训练
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)
E=1
@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):

  # Mapping function to apply to each batch.
  @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
  def batch_fn(model, batch):
    for _ in range(E):
      model=batch_train(model, batch, learning_rate)
    return model

  return tff.sequence_reduce(all_batches, initial_model, batch_fn)

#设备评估
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval(model, all_batches):
  # TODO(b/120157713): Replace with `tff.sequence_average()` once implemented.
  return tff.sequence_sum(
      tff.sequence_map(
          tff.federated_computation(lambda b: batch_loss(model, b), BATCH_TYPE),
          all_batches))

#联邦评估
SERVER_MODEL_TYPE = tff.type_at_server(MODEL_TYPE)
CLIENT_DATA_TYPE = tff.type_at_clients(LOCAL_DATA_TYPE)
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model, data):
  return tff.federated_mean(
      tff.federated_map(local_eval, [tff.federated_broadcast(model), data]))
  

#联邦训练
SERVER_FLOAT_TYPE = tff.type_at_server(tf.float32)

@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE,
                           CLIENT_DATA_TYPE,tff.type_at_clients(tf.float32))
def federated_train(model, learning_rate, data,w):
  return tff.federated_mean(
      tff.federated_map(local_train, [
          tff.federated_broadcast(model),
          tff.federated_broadcast(learning_rate), data
      ]),w)
  
  
#计算准确率
def accur(model):
  device_index=range(1000)
  device=[]
  device.append({
      'x':
          np.array([mnist_test[0][i].flatten() / 255.0 for i in device_index],
                  dtype=np.float32),
      'y':
          np.array([mnist_test[1][i] for i in device_index], dtype=np.int32)
  })
  predicted_y = tf.nn.softmax(
        tf.matmul(device[0]['x'], model['weights']) + model['bias'])
  p=[]
  for i in predicted_y:
    max=-1
    flag=0
    for j in range(1,10):
      if i[j]>max:
        max=i[j]
        flag=j
    p.append(flag)
  cnt=0
  for i in range(1000):
    if p[i]==device[0]['y'][i]:
      cnt+=1
  return cnt/1000

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
initial_model = collections.OrderedDict(
    weights=np.zeros([784, 10], dtype=np.float32),
    bias=np.zeros([10], dtype=np.float32))
#设置权值
weight_list = [1.9,0.1]
for _ in range(98):
  weight_list.append(1)
weight_list = [x/100 for x in weight_list]
model = initial_model
for round_num in range(300):
  learning_rate = 0.1 /(1+round_num)
  model = federated_train(model, learning_rate, federated_train_data,weight_list)
  loss = federated_eval(model, federated_train_data)
  acc = accur(model)
  print('round {}, loss={}, acc={}%'.format(round_num, loss, acc*100))


round 0, loss=10.372981071472168, acc=70.39999999999999%
round 1, loss=9.729572296142578, acc=71.8%
round 2, loss=9.276639938354492, acc=71.8%
round 3, loss=8.937175750732422, acc=71.8%
round 4, loss=8.671344757080078, acc=71.8%
round 5, loss=8.455917358398438, acc=71.8%
round 6, loss=8.27657699584961, acc=71.89999999999999%
round 7, loss=8.124074935913086, acc=71.8%
round 8, loss=7.992163181304932, acc=71.8%
round 9, loss=7.876453399658203, acc=71.8%
round 10, loss=7.773778915405273, acc=71.8%
round 11, loss=7.681778430938721, acc=71.89999999999999%
round 12, loss=7.598653793334961, acc=71.89999999999999%
round 13, loss=7.52301025390625, acc=72.0%
round 14, loss=7.453742027282715, acc=72.0%
round 15, loss=7.389968395233154, acc=72.0%
round 16, loss=7.330965995788574, acc=72.0%
round 17, loss=7.276141166687012, acc=72.0%
round 18, loss=7.224999904632568, acc=72.0%
round 19, loss=7.1771321296691895, acc=71.89999999999999%
round 20, loss=7.132178783416748, acc=71.8%
round 21, loss=7.0898

round 0, loss=21.60552406311035
round 1, loss=20.365678787231445
round 2, loss=19.27480125427246
round 3, loss=18.31110954284668
round 4, loss=17.457256317138672

不改权值的结果：
```
round 0, loss=10.36648941040039, acc=70.3%
round 1, loss=9.720335960388184, acc=71.39999999999999%
round 2, loss=9.265982627868652, acc=71.5%
round 3, loss=8.925678253173828, acc=71.5%
round 4, loss=8.659313201904297, acc=71.6%
round 5, loss=8.443524360656738, acc=71.5%
round 6, loss=8.263936042785645, acc=71.7%
round 7, loss=8.1112642288208, acc=71.6%
round 8, loss=7.979235649108887, acc=71.6%
round 9, loss=7.8634490966796875, acc=71.6%
round 10, loss=7.760721206665039, acc=71.7%
round 11, loss=7.668695449829102, acc=71.7%
round 12, loss=7.585556507110596, acc=71.8%
round 13, loss=7.509913921356201, acc=71.8%
round 14, loss=7.4406561851501465, acc=71.8%
round 15, loss=7.376895904541016, acc=71.8%
round 16, loss=7.317915439605713, acc=71.8%
round 17, loss=7.263117790222168, acc=71.8%
round 18, loss=7.212005615234375, acc=71.8%
round 19, loss=7.164167404174805, acc=71.8%
round 20, loss=7.119253635406494, acc=71.7%
round 21, loss=7.076962471008301, acc=71.89999999999999%
round 22, loss=7.037031650543213, acc=72.0%
round 23, loss=6.999242782592773, acc=72.0%
round 24, loss=6.963403701782227, acc=72.0%
round 25, loss=6.929336071014404, acc=72.0%
round 26, loss=6.896897792816162, acc=72.0%
round 27, loss=6.865954399108887, acc=72.0%
round 28, loss=6.836390495300293, acc=72.0%
round 29, loss=6.808094501495361, acc=72.0%
round 30, loss=6.780981063842773, acc=72.0%
round 31, loss=6.754961013793945, acc=72.0%
round 32, loss=6.729955673217773, acc=72.0%
round 33, loss=6.705905914306641, acc=72.0%
round 34, loss=6.682745933532715, acc=72.0%
round 35, loss=6.660414218902588, acc=72.0%
round 36, loss=6.638864040374756, acc=72.0%
round 37, loss=6.618048667907715, acc=72.0%
round 38, loss=6.5979228019714355, acc=72.1%
round 39, loss=6.578448295593262, acc=72.1%
round 40, loss=6.559586048126221, acc=72.1%
round 41, loss=6.541306972503662, acc=72.1%
round 42, loss=6.523574352264404, acc=72.1%
round 43, loss=6.5063652992248535, acc=72.1%
round 44, loss=6.489650726318359, acc=72.2%
round 45, loss=6.473405838012695, acc=72.2%
round 46, loss=6.457608699798584, acc=72.2%
round 47, loss=6.442233085632324, acc=72.2%
round 48, loss=6.427267551422119, acc=72.2%
round 49, loss=6.412689208984375, acc=72.1%
round 50, loss=6.398477077484131, acc=72.1%
round 51, loss=6.384622573852539, acc=72.1%
round 52, loss=6.371103286743164, acc=72.1%
round 53, loss=6.357907772064209, acc=72.1%
round 54, loss=6.345022201538086, acc=72.2%
round 55, loss=6.332433700561523, acc=72.2%
round 56, loss=6.320131301879883, acc=72.2%
round 57, loss=6.308100700378418, acc=72.2%
round 58, loss=6.296338081359863, acc=72.2%
round 59, loss=6.284825325012207, acc=72.3%
round 60, loss=6.273555755615234, acc=72.3%
round 61, loss=6.262521266937256, acc=72.3%
round 62, loss=6.251714706420898, acc=72.3%
round 63, loss=6.241124153137207, acc=72.3%
round 64, loss=6.230745315551758, acc=72.3%
round 65, loss=6.220568656921387, acc=72.3%
round 66, loss=6.210587978363037, acc=72.3%
round 67, loss=6.200796127319336, acc=72.3%
round 68, loss=6.191188335418701, acc=72.3%
round 69, loss=6.181758403778076, acc=72.3%
round 70, loss=6.172497749328613, acc=72.1%
round 71, loss=6.16340446472168, acc=72.1%
round 72, loss=6.154470920562744, acc=72.1%
round 73, loss=6.145693778991699, acc=72.2%
round 74, loss=6.137065887451172, acc=72.2%
round 75, loss=6.128585338592529, acc=72.3%
round 76, loss=6.120245456695557, acc=72.3%
round 77, loss=6.1120452880859375, acc=72.3%
round 78, loss=6.103979110717773, acc=72.3%
round 79, loss=6.09604024887085, acc=72.3%
round 80, loss=6.088228702545166, acc=72.3%
round 81, loss=6.080538749694824, acc=72.3%
round 82, loss=6.072969913482666, acc=72.3%
round 83, loss=6.065515518188477, acc=72.3%
round 84, loss=6.058176040649414, acc=72.3%
round 85, loss=6.050944805145264, acc=72.3%
round 86, loss=6.043820858001709, acc=72.3%
round 87, loss=6.036800384521484, acc=72.3%
round 88, loss=6.029882907867432, acc=72.3%
round 89, loss=6.023061752319336, acc=72.3%
round 90, loss=6.016338348388672, acc=72.3%
round 91, loss=6.009708881378174, acc=72.3%
round 92, loss=6.003171920776367, acc=72.3%
round 93, loss=5.996725082397461, acc=72.3%
round 94, loss=5.9903645515441895, acc=72.3%
round 95, loss=5.984086513519287, acc=72.3%
round 96, loss=5.977894306182861, acc=72.3%
round 97, loss=5.971782684326172, acc=72.3%
round 98, loss=5.965747833251953, acc=72.3%
round 99, loss=5.959793567657471, acc=72.3%
round 100, loss=5.953916072845459, acc=72.3%
round 101, loss=5.948107719421387, acc=72.3%
round 102, loss=5.942374229431152, acc=72.3%
round 103, loss=5.936712741851807, acc=72.3%
round 104, loss=5.9311203956604, acc=72.3%
round 105, loss=5.925593852996826, acc=72.3%
round 106, loss=5.920135974884033, acc=72.3%
round 107, loss=5.914740085601807, acc=72.3%
round 108, loss=5.9094109535217285, acc=72.3%
round 109, loss=5.904142379760742, acc=72.3%
round 110, loss=5.898932933807373, acc=72.3%
round 111, loss=5.8937859535217285, acc=72.3%
round 112, loss=5.888695240020752, acc=72.39999999999999%
round 113, loss=5.883661270141602, acc=72.39999999999999%
round 114, loss=5.8786845207214355, acc=72.39999999999999%
round 115, loss=5.873762607574463, acc=72.39999999999999%
round 116, loss=5.868896007537842, acc=72.5%
round 117, loss=5.864081382751465, acc=72.5%
round 118, loss=5.859317779541016, acc=72.5%
round 119, loss=5.8546037673950195, acc=72.5%
round 120, loss=5.849942684173584, acc=72.5%
round 121, loss=5.845328330993652, acc=72.5%
round 122, loss=5.840763568878174, acc=72.5%
round 123, loss=5.836244106292725, acc=72.5%
round 124, loss=5.831772327423096, acc=72.5%
round 125, loss=5.827345371246338, acc=72.5%
round 126, loss=5.822963237762451, acc=72.5%
round 127, loss=5.818624496459961, acc=72.5%
round 128, loss=5.8143310546875, acc=72.5%
round 129, loss=5.8100762367248535, acc=72.5%
round 130, loss=5.805863857269287, acc=72.5%
round 131, loss=5.801693916320801, acc=72.5%
round 132, loss=5.7975640296936035, acc=72.5%
round 133, loss=5.79347038269043, acc=72.5%
round 134, loss=5.789417743682861, acc=72.5%
round 135, loss=5.785403251647949, acc=72.5%
round 136, loss=5.781425952911377, acc=72.5%
round 137, loss=5.7774858474731445, acc=72.5%
round 138, loss=5.7735819816589355, acc=72.5%
round 139, loss=5.769712448120117, acc=72.5%
round 140, loss=5.765878200531006, acc=72.6%
round 141, loss=5.762081146240234, acc=72.6%
round 142, loss=5.7583136558532715, acc=72.6%
round 143, loss=5.754583358764648, acc=72.6%
round 144, loss=5.750883102416992, acc=72.6%
round 145, loss=5.747217178344727, acc=72.6%
round 146, loss=5.743582248687744, acc=72.6%
round 147, loss=5.7399773597717285, acc=72.6%
round 148, loss=5.736404895782471, acc=72.6%
round 149, loss=5.732864856719971, acc=72.6%
round 150, loss=5.729351043701172, acc=72.6%
round 151, loss=5.7258687019348145, acc=72.6%
round 152, loss=5.722413539886475, acc=72.6%
round 153, loss=5.718986988067627, acc=72.6%
round 154, loss=5.7155914306640625, acc=72.6%
round 155, loss=5.712220668792725, acc=72.6%
round 156, loss=5.7088799476623535, acc=72.6%
round 157, loss=5.705564498901367, acc=72.6%
round 158, loss=5.702276229858398, acc=72.6%
round 159, loss=5.699013710021973, acc=72.6%
round 160, loss=5.695777416229248, acc=72.5%
round 161, loss=5.69256591796875, acc=72.5%
round 162, loss=5.689380645751953, acc=72.5%
round 163, loss=5.68621826171875, acc=72.5%
round 164, loss=5.683082103729248, acc=72.5%
round 165, loss=5.679969310760498, acc=72.5%
round 166, loss=5.676881313323975, acc=72.5%
round 167, loss=5.673816680908203, acc=72.5%
round 168, loss=5.670772075653076, acc=72.5%
round 169, loss=5.667752742767334, acc=72.5%
round 170, loss=5.664754867553711, acc=72.5%
round 171, loss=5.6617817878723145, acc=72.5%
round 172, loss=5.6588263511657715, acc=72.5%
round 173, loss=5.655895233154297, acc=72.5%
round 174, loss=5.65298318862915, acc=72.5%
round 175, loss=5.650094985961914, acc=72.6%
round 176, loss=5.647225856781006, acc=72.6%
round 177, loss=5.6443772315979, acc=72.6%
round 178, loss=5.6415510177612305, acc=72.6%
round 179, loss=5.638741493225098, acc=72.6%
round 180, loss=5.6359543800354, acc=72.6%
round 181, loss=5.633185386657715, acc=72.6%
round 182, loss=5.630435943603516, acc=72.6%
round 183, loss=5.627706527709961, acc=72.6%
round 184, loss=5.624994277954102, acc=72.6%
round 185, loss=5.6222991943359375, acc=72.6%
round 186, loss=5.619626522064209, acc=72.6%
round 187, loss=5.616968154907227, acc=72.6%
round 188, loss=5.614329814910889, acc=72.6%
round 189, loss=5.611709117889404, acc=72.6%
round 190, loss=5.609104156494141, acc=72.6%
round 191, loss=5.606518745422363, acc=72.6%
round 192, loss=5.603947639465332, acc=72.6%
round 193, loss=5.6013946533203125, acc=72.6%
round 194, loss=5.598859786987305, acc=72.6%
round 195, loss=5.596339225769043, acc=72.6%
round 196, loss=5.593835353851318, acc=72.6%
round 197, loss=5.591347694396973, acc=72.6%
round 198, loss=5.588876724243164, acc=72.6%
round 199, loss=5.586421012878418, acc=72.6%
round 200, loss=5.583980083465576, acc=72.6%
round 201, loss=5.581554412841797, acc=72.6%
round 202, loss=5.579144477844238, acc=72.6%
round 203, loss=5.576749801635742, acc=72.6%
round 204, loss=5.57436990737915, acc=72.6%
round 205, loss=5.5720014572143555, acc=72.6%
round 206, loss=5.569650173187256, acc=72.6%
round 207, loss=5.567317008972168, acc=72.6%
round 208, loss=5.5649919509887695, acc=72.6%
round 209, loss=5.562684535980225, acc=72.6%
round 210, loss=5.56038761138916, acc=72.6%
round 211, loss=5.558109283447266, acc=72.6%
round 212, loss=5.555840492248535, acc=72.6%
round 213, loss=5.553586483001709, acc=72.6%
round 214, loss=5.5513458251953125, acc=72.6%
round 215, loss=5.549118518829346, acc=72.6%
round 216, loss=5.546905040740967, acc=72.6%
round 217, loss=5.544701099395752, acc=72.6%
round 218, loss=5.542513847351074, acc=72.6%
round 219, loss=5.540337085723877, acc=72.6%
round 220, loss=5.538171291351318, acc=72.6%
round 221, loss=5.5360212326049805, acc=72.6%
round 222, loss=5.533881664276123, acc=72.6%
round 223, loss=5.531755447387695, acc=72.6%
round 224, loss=5.529639720916748, acc=72.6%
round 225, loss=5.527536392211914, acc=72.6%
round 226, loss=5.525442123413086, acc=72.6%
round 227, loss=5.52336311340332, acc=72.6%
round 228, loss=5.521296977996826, acc=72.6%
round 229, loss=5.519237041473389, acc=72.6%
round 230, loss=5.517190933227539, acc=72.6%
round 231, loss=5.515157699584961, acc=72.6%
round 232, loss=5.513134002685547, acc=72.6%
round 233, loss=5.511120796203613, acc=72.7%
round 234, loss=5.509117603302002, acc=72.7%
round 235, loss=5.507127285003662, acc=72.7%
round 236, loss=5.505147933959961, acc=72.7%
round 237, loss=5.503175258636475, acc=72.7%
round 238, loss=5.501216411590576, acc=72.89999999999999%
round 239, loss=5.499266147613525, acc=72.89999999999999%
round 240, loss=5.49732780456543, acc=72.89999999999999%
round 241, loss=5.49539852142334, acc=72.89999999999999%
round 242, loss=5.493480205535889, acc=72.89999999999999%
round 243, loss=5.491570949554443, acc=72.89999999999999%
round 244, loss=5.489671230316162, acc=72.89999999999999%
round 245, loss=5.4877824783325195, acc=72.89999999999999%
round 246, loss=5.485902786254883, acc=72.89999999999999%
round 247, loss=5.4840312004089355, acc=72.89999999999999%
round 248, loss=5.482170581817627, acc=72.89999999999999%
round 249, loss=5.480320453643799, acc=72.89999999999999%
round 250, loss=5.47847843170166, acc=72.89999999999999%
round 251, loss=5.476645469665527, acc=72.89999999999999%
round 252, loss=5.474822521209717, acc=72.89999999999999%
round 253, loss=5.473005771636963, acc=72.89999999999999%
round 254, loss=5.4712018966674805, acc=72.89999999999999%
round 255, loss=5.469404220581055, acc=72.89999999999999%
round 256, loss=5.467617988586426, acc=72.89999999999999%
round 257, loss=5.4658379554748535, acc=72.89999999999999%
round 258, loss=5.4640679359436035, acc=72.89999999999999%
round 259, loss=5.462305068969727, acc=72.89999999999999%
round 260, loss=5.460551738739014, acc=72.89999999999999%
round 261, loss=5.458808422088623, acc=72.89999999999999%
round 262, loss=5.457070827484131, acc=72.89999999999999%
round 263, loss=5.4553422927856445, acc=72.89999999999999%
round 264, loss=5.453622341156006, acc=72.89999999999999%
round 265, loss=5.451910495758057, acc=72.89999999999999%
round 266, loss=5.4502081871032715, acc=72.89999999999999%
round 267, loss=5.448512554168701, acc=72.89999999999999%
round 268, loss=5.446822643280029, acc=72.89999999999999%
round 269, loss=5.445144176483154, acc=72.89999999999999%
round 270, loss=5.443472385406494, acc=72.89999999999999%
round 271, loss=5.441807746887207, acc=72.89999999999999%
round 272, loss=5.440150737762451, acc=72.89999999999999%
round 273, loss=5.438502311706543, acc=72.89999999999999%
round 274, loss=5.436861515045166, acc=72.89999999999999%
round 275, loss=5.435227870941162, acc=72.89999999999999%
round 276, loss=5.433599948883057, acc=72.89999999999999%
round 277, loss=5.431982040405273, acc=72.89999999999999%
round 278, loss=5.430369853973389, acc=72.89999999999999%
round 279, loss=5.428764820098877, acc=72.89999999999999%
round 280, loss=5.427168369293213, acc=73.0%
round 281, loss=5.425579071044922, acc=73.0%
round 282, loss=5.423994064331055, acc=73.0%
round 283, loss=5.422417163848877, acc=73.0%
round 284, loss=5.420848846435547, acc=73.0%
round 285, loss=5.419287204742432, acc=73.0%
round 286, loss=5.417731761932373, acc=73.0%
round 287, loss=5.4161834716796875, acc=73.0%
round 288, loss=5.4146409034729, acc=73.2%
round 289, loss=5.4131059646606445, acc=73.2%
round 290, loss=5.411578178405762, acc=73.2%
round 291, loss=5.410055637359619, acc=73.2%
round 292, loss=5.40854024887085, acc=73.2%
round 293, loss=5.40703010559082, acc=73.2%
round 294, loss=5.405528545379639, acc=73.2%
round 295, loss=5.404033184051514, acc=73.2%
round 296, loss=5.402544021606445, acc=73.2%
round 297, loss=5.401060104370117, acc=73.2%
round 298, loss=5.399583339691162, acc=73.2%
round 299, loss=5.398112773895264, acc=73.2%
```

改变权值(0.19,0.01)后的结果
```
round 0, loss=10.372981071472168, acc=70.39999999999999%
round 1, loss=9.729572296142578, acc=71.8%
round 2, loss=9.276639938354492, acc=71.8%
round 3, loss=8.937175750732422, acc=71.8%
round 4, loss=8.671344757080078, acc=71.8%
round 5, loss=8.455917358398438, acc=71.8%
round 6, loss=8.27657699584961, acc=71.89999999999999%
round 7, loss=8.124074935913086, acc=71.8%
round 8, loss=7.992163181304932, acc=71.8%
round 9, loss=7.876453399658203, acc=71.8%
round 10, loss=7.773778915405273, acc=71.8%
round 11, loss=7.681778430938721, acc=71.89999999999999%
round 12, loss=7.598653793334961, acc=71.89999999999999%
round 13, loss=7.52301025390625, acc=72.0%
round 14, loss=7.453742027282715, acc=72.0%
round 15, loss=7.389968395233154, acc=72.0%
```